## Computational Chemistry for Experimentalists
## Module 2: Properties of Methane 

We introduce computational chemistry tools by looking at some properties of the methane molecule. 

This first block imports all of the necessary Python modules. If these aren't installed, this will fail. 

In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
from pyscf import gto,scf
from pyscf.tools import cubegen 
import py3Dmol
import numpy 

Use the RDKit force field to prepare an initial guess geometry of methane 

In [ ]:
m=Chem.MolFromSmiles('C')
m2=Chem.AddHs(m)
AllChem.EmbedMolecule(m2)
AllChem.MMFFOptimizeMolecule(m2)

Visualize the initial guess geometry using a ball-and-stick convention. Credit to https://www.andersle.no/posts/2022/rdkitandpy3dmol/rdkit.html. It is always good to double-check that your inputs are chemically reasonable. 

In [ ]:
mb=Chem.MolToMolBlock(m2)
p=py3Dmol.view(width=400,height=400)
p.addModel(mb,'sdf')
p.setStyle({'stick':{},'sphere':{"scale":0.3}})
p.zoomTo()
p.show()

Convert the RDKit geometry into a PySCF molecule, and do a molecular orbital calculation at the RDKit geometry.
Credit to https://www.valencekjell.com/posts/2022-06-30-visualizing-atomic/ 

In [ ]:
elements = [atom.GetSymbol() for atom in m2.GetAtoms()]
coordinates = m2.GetConformer().GetPositions()
atoms = [(element, coordinate) for element, coordinate in zip(elements, coordinates)]

pyscf_mole = gto.Mole(basis="3-21g")
pyscf_mole.atom = atoms
pyscf_mole.build();

mf=scf.RHF(pyscf_mole)
mf.kernel() 
print('Total energy from Hartree-Fock molecular orbital calculation: %.6f Hartree'%(mf.e_tot))
print('Number of occupied orbitals: %d'%(pyscf_mole.nelec[0]))


Process the RDKit guess coordinates to determine the guess structure's C-H bond length. The experimental C-H bond length is 1.057 Angstrom, as quoted in the NIST CCCBDB https://cccbdb.nist.gov/exp2x.asp?casno=74828&charge=0 . The coordinates array above is a 5x3 array of the x,y,z coordinates of each atom 

In [ ]:
print('List of atom types: ',elements)
print('Shape of coordinate array: ',coordinates.shape)
VCH=coordinates[0]-coordinates[1] # Vector connecting atoms C and H 
#print(VCH)
RCH=(numpy.dot(VCH,VCH))**0.5 # Length of a vector v is sqrt(v.v)
print('RDKit predicted C-H bond length: %.3f Angstrom'%(RCH))

Print the occupied and virtual orbital energies from array mf.mo_energies. The indexing starts at zero, so mf.mo_energies[0] is the C 1s orbital, mf.mo_energies[1] is the first valence orbital, and so on. Experimentally, methane's first ionization potential is 12.610 eV (CCCBDB) and its core ionization potential is 290.707 eV (L Asplund et al 1985 J. Phys. B: Atom. Mol. Phys. 18 1569). The computed orbital energies don't perfectly match the experimental ionization potentials, for reasons we'll discuss later. 

In [ ]:
a2ev=27.211 # Hartree atomic unit to electron-volt conversion 
print('Orbital energies \nOrbital  Occupied? Energy (eV)')
for i in range(pyscf_mole.nao):
    st='No'
    if(i<pyscf_mole.nelec[0]):
        st='Yes'
    print('%2d   %3s %12.2f '%(i,st,a2ev*mf.mo_energy[i]))

Print some other properties available from the MO calculation 

In [ ]:
x=mf.analyze()

Generate volumetric data from all molecular orbitals, and save in the current directory as cube files methane_mo_*.cube 

In [ ]:
for i in range(mf.mo_coeff.shape[1]):
    cubegen.orbital(pyscf_mole, f'methane_mo_{i+1:02d}.cube', mf.mo_coeff[:,i],  nx=60, ny=60, nz=60)

Visualize the three degenerate highest-occupied molecular orbitals. Credit to 

https://www.insilicochemistry.io/tutorials/foundations/chemistry-visualization-with-py3dmol

In [ ]:
p = py3Dmol.view(width=600,height=200,viewergrid=(1,3))
#p.show()
for i in range(3):
    p.addModel(Chem.MolToMolBlock(m2), 'sdf',viewer=(0,i))
    p.setStyle({'stick':{},'sphere':{"scale":0.3}},viewer=(0,i))
    with open(f"./methane_mo_{i+3:02d}.cube") as f:
        mo1 = f.read()
    p.addVolumetricData(mo1, "cube", {'isoval': -0.02, 'color': "red", 'opacity': 0.65},viewer=(0,i))
    p.addVolumetricData(mo1, "cube", {'isoval': 0.02, 'color': "blue", 'opacity': 0.65},viewer=(0,i))
#p.zoomTo()
#p.update()
p.render()


Visualize the nondegenerate lowest-unoccupied molecular orbital at three different cutoffs 

In [ ]:
with open(f"./methane_mo_06.cube") as f:
    mo1 = f.read()
p = py3Dmol.view(width=600,height=200,viewergrid=(1,3))
cutoffs=[0.05,0.03,0.01]
for i in range(3):
    cut=cutoffs[i]
    p.addModel(Chem.MolToMolBlock(m2), 'sdf',viewer=(0,i))
    p.setStyle({'stick':{},'sphere':{"scale":0.3}},viewer=(0,i))
    p.addVolumetricData(mo1, "cube", {'isoval': -cut, 'color': "red", 'opacity': 0.45},viewer=(0,i))
    p.addVolumetricData(mo1, "cube", {'isoval': cut, 'color': "blue", 'opacity': 0.65},viewer=(0,i))
#p.zoomTo()
#p.update()
p.render()

Your assignment for this module is as follows. 

Part 1: Explain whether the partial atomic charges computed with mf.analyze() make chemical sense. 
    
Part 2: Visualize the four highest-occupied molecular orbitals of methane. One of them should look qualitatively different from the other three. Explain this in terms of general chemistry MO theory.

Part 3: Use the orbital energies computed with mf.analyze() to predict the electron affinity and UV/vis excitation energy of methane. (You already used these to predict methane's ionization potential.) Compare your results to available experiments. You should find that the results are within about a factor of 2 of experiment. Not accurate enough to model real chemistry, but not unphysical. 